# About these notebooks

These three notebooks can be run to replicate the dataset, XGBoost model and results from the tree-based structure prediction. The notebooks save and load data in local folders in the Metalfinder_restructured folder. The first notebook simulates the metal clusters and PDFs. The second notebook trains an XGBoost model. The third notebook tests the XGBoost model and here the model can be used to predict the structure of experimental PDFs.

# Simulate Data

This notebook simulates clusters and pair distribution function datasets

**How to use:** Run the cells underneath from top to bottom 

The first cell imports packages and functions from the backend. The python packages Atomic Simulation Environment (ASE)*(1)* and DiffPy-CMI*(2)* are used to simulate data.

The second cell simulates metal clusters of seven different structure types (MMNPs) and saves them in a custom folder in /xyz_files/. 

The third cell simulates Atomic Pair Distribution functions from the metal clusters and saves them in a custom folder in /PDF_datasets/. This dataset can be loaded in "2_Training.ipynb", where a Gradient Boosted Decision Tree model can be trained to predict the MMNPs from a PDF.

# Import packages and functions from backend

In [2]:
import sys, os, os.path, h5py, time, shutil, mendeleev
import pandas as pd
import numpy as np

from os import walk
from tqdm import tqdm
from shutil import copyfile
from scipy import spatial

from multiprocessing import Pool
from diffpy.srreal.pdfcalculator import DebyePDFCalculator, PDFCalculator
from diffpy.Structure import loadStructure, Structure, Lattice, Atom

from diffpy.Structure import Structure
from diffpy.Structure import Atom
from diffpy.Structure import loadStructure, Structure, Lattice

from ase.data import covalent_radii, atomic_numbers, chemical_symbols
from ase.cluster.cubic import FaceCenteredCubic, BodyCenteredCubic, SimpleCubic
from ase.lattice.hexagonal import Hexagonal, HexagonalClosedPacked, Graphite
from ase.cluster.decahedron import Decahedron
from ase.cluster.icosahedron import Icosahedron
from ase.cluster.octahedron import Octahedron

sys.path.append("Backend")

from simulate_data import PDF_generator, sort_filenames, make_df_parallelized, PDF_from_index, reduce_mem_usage, save_PDFs, make_data, make_data_SC_FCC_BCC, make_data_HCP, make_data_Icosahedron, make_data_Decahedron, make_data_Octahedron, new_structure_checker, remove_same_structures, structure_maker, remove_dub, simulate_clusters#, PDF_from_index_test   

## Simulate metal clusters

This simulates metal clusters of different sizes and structure types, then checks for and deletes duplicates. The metal clusters are saved in a specified folder_name in the "xyz_files/" folder.

In [ ]:
folder_name = "natoms200"
simulate_clusters(folder_name, maximum_atoms = 200, minimum_atoms = 6)

## Simulate PDF dataset from metal clusters

This simulates atomic pair distribution functions from each metallic nanoparticle cluster. The size of the dataset can be controlled with train_multip and valtest_multip. train_multip determines how many pair distributions are simulated from each metal cluster in the training set. valtest_multip is similar, but with the validation and test set. In the article, train_multip = 100 and valtest_multip = 15 is used. n_threads specifies how many CPU cores are used.

In [ ]:
folder_name = "natoms200"
xyz_path = "xyz_files/" + folder_name + "/"

threads = 4
X_train, y_train, X_val, y_val, X_test, y_test = make_df_parallelized(sorted_filenames_flat, train_multip = 2, valtest_multip = 1, n_processes = threads, xyz_path = xyz_path)

## Save PDF dataset

This saves the datasets in the specified folder in "PDF_datasets/". These datasets can be loaded in 2_Training.ipynb where an XGBoost model can be trained on the datasets.

In [ ]:
folder_name = "natoms200"
save_PDFs(folder_name, X_train, y_train, X_val, y_val, X_test, y_test)

### References
(1) Larsen, A. H.; Mortensen, J. J.; Blomqvist, J.; Castelli, I. E.; Christensen, R.; Dułak, M.; Friis, J.; Groves, M. N.; Hammer, B.; Hargus, C. The atomic simulation environment—a Python library for working with atoms. Journal of Physics: Condensed Matter 2017, 29 (27), 273002.

(2) Billinge, S.; Juhas, P.; Farrow, C.; McKerns, M. DiffPy-CMI-Python libraries for Complex Modeling Initiative; Brookhaven National Lab.(BNL), Upton, NY (United States), 2014.